In [56]:
!pip install -e .

Obtaining file:///Users/ydchen/MBDS_FYP/FYP_v3
  Preparing metadata (setup.py) ... done
  Attempting uninstall: myproject
    Found existing installation: myproject 1.0
    Uninstalling myproject-1.0:
      Successfully uninstalled myproject-1.0
  Running setup.py develop for myproject


In [57]:
from Crypto.Hash import keccak
def obtain_hash_event(event: str) -> str:
    k = keccak.new(digest_bits=256)
    k.update(bytes(event, encoding='utf-8'))
    return '0x'+k.hexdigest()
# obtain_hash_event('PairCreated(address,address,address,uint256)')

In [58]:
# Transfer_hash = obtain_hash_event('Transfer(address,address,uint256)')
# Burn_hash = obtain_hash_event('Burn(address,uint256,uint256,address)')
# Mint_hash = obtain_hash_event("Mint(address,uint256,uint256)")
# Sync_hash = obtain_hash_event('Sync(uint112,uint112)')

In [59]:
import time
import pandas as pd
import requests
import shared
shared.init()
import json
from web3 import Web3, HTTPProvider
from hexbytes import HexBytes
from web3.datastructures import AttributeDict
from Utils.eth_utils import  obtain_hash_event

def connect_to_web3():
    """
    Connect to Web3 server.
    Args:
    Returns:
      res: Boolean indicating that connection was succeed or not.
      web3: Web3 Object
    """
    web3 = Web3(HTTPProvider('https://polygon-mainnet.infura.io/v3/d6243bb783b44485ad6636b6c3411377'))
    res = web3.isConnected()
    return res, web3

from_block = 22757547
response = [0] * 1000
block_number = []
_, web3 = connect_to_web3()
contract = web3.eth.contract(shared.UNISWAP_V2_FACTORY, abi="""[{"inputs":[{"internalType":"address","name":"_feeToSetter","type":"address"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"token0","type":"address"},{"indexed":true,"internalType":"address","name":"token1","type":"address"},{"indexed":false,"internalType":"address","name":"pair","type":"address"},{"indexed":false,"internalType":"uint256","name":"","type":"uint256"}],"name":"PairCreated","type":"event"},{"constant":true,"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"allPairs","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"allPairsLength","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"tokenA","type":"address"},{"internalType":"address","name":"tokenB","type":"address"}],"name":"createPair","outputs":[{"internalType":"address","name":"pair","type":"address"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"feeTo","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"feeToSetter","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"address","name":"","type":"address"}],"name":"getPair","outputs":[{"internalType":"address","name":"","type":"address"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"_feeTo","type":"address"}],"name":"setFeeTo","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"internalType":"address","name":"_feeToSetter","type":"address"}],"name":"setFeeToSetter","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"}]""")

topic = obtain_hash_event('PairCreated(address,address,address,uint256)')
# topic = obtain_hash_event('PoolCreated(address,address,uint24,int24,address')


tokens0, tokens1, pairs, block_numbers, timestamps = [], [], [], [], []
cont = 1000
while len(response) >= 1000:
    try:
        print("Pools trobades", cont)
        cont += 1000
        apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
        endpoint = "https://api.polygonscan.com/api?module=logs" \
                   "&action=getLogs" \
                   f"&fromBlock={from_block}" \
                   f"&toBlock={shared.BLOCKSTUDY}" \
                   f"&address={shared.SUSHISWAP_FACTORY}" \
                   f"&topic0={topic}" \
                   f"&apikey={apikey}"

        response = json.loads(requests.get(endpoint).text)['result']

        # print(response)
        receipts = []
        for event in response:
            receipts.append(AttributeDict({
                'blockNumber': int(event['blockNumber'], 16),
                'contractAddress': event['address'],
                'topics': [HexBytes(topic) for topic in event['topics']],
                'transactionHash': HexBytes(event['transactionHash']),
                'data': event['data'],
                'logIndex': 0,
                'transactionIndex': event['transactionIndex'],
                'address': event['address'],
                'blockHash': HexBytes(event['transactionHash'])
            }))
        log_dict = {'logs': receipts}
        # print(log_dict)
        decoded_logs = contract.events.PairCreated().processReceipt(log_dict)
        # print(decoded_logs)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            tokens0.append(log['args']["token0"])
            tokens1.append(log['args']['token1'])
            pairs.append(log['args']['pair'])
            #fees.append(log['args']['fee'])
            timestamps.append(int(response[k]['timeStamp'], 16))
        from_block = int(response[-1]['blockNumber'], 16) + 1

    except Exception as err:
        print("error", err)
        time.sleep(2)

pd.DataFrame({'pair': pairs, 'token0': tokens0, 'token1': tokens1, 'block_number': block_numbers, 'timestamp': timestamps}).to_csv(
    "./data/uniswap_v2/polygon_pools.csv", index=False)

Pools trobades 1000
Pools trobades 2000


In [60]:
import os
import time
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(pool, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={pool}" \
               f"&topic0={sync_hash}" \
               f"&apikey={apikey}"

    return json.loads(requests.get(endpoint).text)['result']

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Sync().processReceipt(log_dict)

sync_hash = obtain_hash_event('Sync(uint112,uint112)')
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
tokens_and_pools = pd.read_csv("./data/uniswap_v2/polygon_pools.csv", index_col='pair')
pools = tokens_and_pools.index.tolist()

contract = web3.eth.contract("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D", abi=shared.ABI_POOL)
done_pools = [pool.split(".csv")[0] for pool in os.listdir("./data/uniswap_v2/sync")]

for i, pool in enumerate(pools):
    if pool in done_pools:
        continue
    response = [0] * 1000

    print(pool, i, len(pools))
    timestamps, block_numbers, reserves0, reserves1 = [], [], [], []
    from_block, first_timestamp, current_timestamp = 0, 0, 0
    cont = 0

    while len(response) >= 1000 and current_timestamp - first_timestamp < shared.WEEK:
        response = get_response(pool, from_block)
        while response == None or "Max rate limit" in response:
            time.sleep(20)
            response = get_response(pool, from_block)
            if response != None:
                break
        if len(response) == 0:
            continue
        try:
            decoded_logs = clean_response(response)
        except:
            print(pool, i, len(pools))
            print(response)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            reserves0.append(log['args']["reserve0"])
            reserves1.append(log['args']["reserve1"])
            timestamps.append(int(response[k]["timeStamp"], 16))
        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)

    pd.DataFrame({'block_number': block_numbers, 'reseve0': reserves0 , 'reserve1': reserves1, 'timestamps': timestamps}).to_csv(
        f"./data/uniswap_v2/sync/{pool}.csv", index=False)

0x13bd20ba647E2ed12622851F652a5267869A8c62 0 1459
0x531eB8f8E8e626cCFaB2BAA38d14A45096a50180 1 1459
0x103C0675054D31FD081f53a15FE108c5B2A2f456 2 1459
0x2d8Db3B54bc4c08B295816a30ad74b01109e4e9C 3 1459
0x61637f75DCecEc29e11D73Bfa3ac698Dd5Ef8a0f 4 1459
0xfA69bF4fA125ef9Cc74945EAc914150e5b3cF3E0 5 1459
0xb874c2403391fc1571f778fb6fa9A909eb0E79b7 6 1459
0x4f414E6489fD19bBb50C1Bc6C403E19D0dC71Fd2 7 1459
0x1A3713dE5Fe72EDe1f5aA08d563Da6c5D4Bc2b9e 8 1459
0x53aDd4C98B2787f690042771Ca8E512a5793E9C9 9 1459
0x46c692346CEDCF46C7c9002680E2645c8577EB3a 10 1459
0x99219f505BfffBE1A58fAF113eBc861b175E41f7 11 1459
0x47F4947a1b44F773EDb4520E170621edf40b2f05 12 1459
0xCD3a618948256F77f794FAe7D30Dea18D5ca51A0 13 1459
0x0b43d8e38d02Ce15D54099A593852824D882cFE2 14 1459
0xE31461B5adCF2174ae53C64dB57C3F59CfED987b 15 1459
0x36B860b863C23a1564aD19F28FD4260443E007a2 16 1459
0x8D04E60fD7cEd96A24e726968F93397C5a467890 17 1459
0x53f858f28Fd9120bB6Ea1b7F5961333EBE734D22 18 1459
0xDeDef8a9ebd5Fb001A596348EB825f0b9EdaF12

In [61]:
#!pip install web3_multicall

In [62]:
import os
import time
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(pool, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={pool}" \
               f"&topic0={sync_hash}" \
               f"&apikey={apikey}"

    return json.loads(requests.get(endpoint).text)['result']

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Swap().processReceipt(log_dict)

sync_hash = "0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822"#obtain_hash_event('Swap(address,unit256,unit256,unit256,unit256,address)')
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
tokens_and_pools = pd.read_csv("./data/uniswap_v2/polygon_pools.csv", index_col='pair')
pools = tokens_and_pools.index.tolist()

contract = web3.eth.contract("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D", abi=shared.ABI_POOL)
done_pools = [pool.split(".csv")[0] for pool in os.listdir("./data/uniswap_v2/swap")]

for i, pool in enumerate(pools):
    if pool in done_pools:
        continue
    response = [0] * 1000

    print(pool, i, len(pools))
    timestamps, block_numbers, sender, to, amount0In, amount1In, amount0Out, amount1Out  = [], [], [], [], [], [], [], []
    from_block, first_timestamp, current_timestamp = 0, 0, 0
    cont = 0

    while len(response) >= 1000 and current_timestamp - first_timestamp < shared.WEEK:
        response = get_response(pool, from_block)
        #print(response)
        while response == None or "Max rate limit" in response:
            time.sleep(20)
            response = get_response(pool, from_block)
            if response != None:
                break
        if len(response) == 0:
            continue
        try:
            decoded_logs = clean_response(response)
        except:
            print(pool, i, len(pools))
            print(response)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            sender.append(log['args']['sender'])
            to.append(log['args']['to'])
            amount0In.append(log['args']['amount0In'])
            amount1In.append(log['args']['amount1In'])
            amount0Out.append(log['args']['amount0Out'])
            amount1Out.append(log['args']['amount1Out'])
            timestamps.append(int(response[k]["timeStamp"], 16))
        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)

    pd.DataFrame({'block_number': block_numbers, 'sender': sender , 'to': to, 'amount0In':amount0In, 'amount1In':amount1In, 'amount0Out':amount0Out, 'amount1Out':amount1Out, 'timestamps': timestamps}).to_csv(
        f"./data/uniswap_v2/swap/{pool}.csv", index=False)

0x13bd20ba647E2ed12622851F652a5267869A8c62 0 1459
0x531eB8f8E8e626cCFaB2BAA38d14A45096a50180 1 1459
0x103C0675054D31FD081f53a15FE108c5B2A2f456 2 1459
0x2d8Db3B54bc4c08B295816a30ad74b01109e4e9C 3 1459
0x61637f75DCecEc29e11D73Bfa3ac698Dd5Ef8a0f 4 1459
0xfA69bF4fA125ef9Cc74945EAc914150e5b3cF3E0 5 1459
0xb874c2403391fc1571f778fb6fa9A909eb0E79b7 6 1459
0x4f414E6489fD19bBb50C1Bc6C403E19D0dC71Fd2 7 1459
0x1A3713dE5Fe72EDe1f5aA08d563Da6c5D4Bc2b9e 8 1459
0x53aDd4C98B2787f690042771Ca8E512a5793E9C9 9 1459
0x46c692346CEDCF46C7c9002680E2645c8577EB3a 10 1459
0x99219f505BfffBE1A58fAF113eBc861b175E41f7 11 1459
0x47F4947a1b44F773EDb4520E170621edf40b2f05 12 1459
0xCD3a618948256F77f794FAe7D30Dea18D5ca51A0 13 1459
0x0b43d8e38d02Ce15D54099A593852824D882cFE2 14 1459
0xE31461B5adCF2174ae53C64dB57C3F59CfED987b 15 1459
0x36B860b863C23a1564aD19F28FD4260443E007a2 16 1459
0x8D04E60fD7cEd96A24e726968F93397C5a467890 17 1459
0x53f858f28Fd9120bB6Ea1b7F5961333EBE734D22 18 1459
0xDeDef8a9ebd5Fb001A596348EB825f0b9EdaF12

In [63]:
import os
import time
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(pool, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={pool}" \
               f"&topic0={sync_hash}" \
               f"&apikey={apikey}"

    return json.loads(requests.get(endpoint).text)['result']

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Mint().processReceipt(log_dict)

sync_hash = obtain_hash_event("Mint(address,uint256,uint256)")
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
tokens_and_pools = pd.read_csv("./data/uniswap_v2/polygon_pools.csv", index_col='pair')
pools = tokens_and_pools.index.tolist()

contract = web3.eth.contract("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D", abi=shared.ABI_POOL)
done_pools = [pool.split(".csv")[0] for pool in os.listdir("./data/uniswap_v2/mint")]

for i, pool in enumerate(pools):
    if pool in done_pools:
        continue
    response = [0] * 1000

    print(pool, i, len(pools))
    timestamps, block_numbers, sender ,amount0, amount1 = [], [], [], [], []
    from_block, first_timestamp, current_timestamp = 0, 0, 0
    cont = 0

    while len(response) >= 1000 and current_timestamp - first_timestamp < shared.WEEK:
        response = get_response(pool, from_block)
        while response == None or "Max rate limit" in response:
            time.sleep(20)
            response = get_response(pool, from_block)
            if response != None:
                break
        if len(response) == 0:
            continue
        try:
            decoded_logs = clean_response(response)
        except:
            print(pool, i, len(pools))
            print(response)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            sender.append(log['args']["sender"])
            amount0.append(log['args']["amount0"])
            amount1.append(log['args']["amount1"])
            timestamps.append(int(response[k]["timeStamp"], 16))
        
        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)

    pd.DataFrame({'block_number': block_numbers, 'sender': sender, 
                  'amount0':amount0,'amount1':amount1,'timestamps': timestamps}).to_csv(f"./data/uniswap_v2/mint/{pool}.csv", index=False)

0x13bd20ba647E2ed12622851F652a5267869A8c62 0 1459
0x531eB8f8E8e626cCFaB2BAA38d14A45096a50180 1 1459
0x103C0675054D31FD081f53a15FE108c5B2A2f456 2 1459
0x2d8Db3B54bc4c08B295816a30ad74b01109e4e9C 3 1459
0x61637f75DCecEc29e11D73Bfa3ac698Dd5Ef8a0f 4 1459
0xfA69bF4fA125ef9Cc74945EAc914150e5b3cF3E0 5 1459
0xb874c2403391fc1571f778fb6fa9A909eb0E79b7 6 1459
0x4f414E6489fD19bBb50C1Bc6C403E19D0dC71Fd2 7 1459
0x1A3713dE5Fe72EDe1f5aA08d563Da6c5D4Bc2b9e 8 1459
0x53aDd4C98B2787f690042771Ca8E512a5793E9C9 9 1459
0x46c692346CEDCF46C7c9002680E2645c8577EB3a 10 1459
0x99219f505BfffBE1A58fAF113eBc861b175E41f7 11 1459
0x47F4947a1b44F773EDb4520E170621edf40b2f05 12 1459
0xCD3a618948256F77f794FAe7D30Dea18D5ca51A0 13 1459
0x0b43d8e38d02Ce15D54099A593852824D882cFE2 14 1459
0xE31461B5adCF2174ae53C64dB57C3F59CfED987b 15 1459
0x36B860b863C23a1564aD19F28FD4260443E007a2 16 1459
0x8D04E60fD7cEd96A24e726968F93397C5a467890 17 1459
0x53f858f28Fd9120bB6Ea1b7F5961333EBE734D22 18 1459
0xDeDef8a9ebd5Fb001A596348EB825f0b9EdaF12

In [64]:
import os
import time
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(pool, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={pool}" \
               f"&topic0={sync_hash}" \
               f"&apikey={apikey}"

    return json.loads(requests.get(endpoint).text)['result']

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Burn().processReceipt(log_dict)

sync_hash = obtain_hash_event('Burn(address,uint256,uint256,address)')
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
tokens_and_pools = pd.read_csv("./data/uniswap_v2/polygon_pools.csv", index_col='pair')
pools = tokens_and_pools.index.tolist()

contract = web3.eth.contract("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D", abi=shared.ABI_POOL)
done_pools = [pool.split(".csv")[0] for pool in os.listdir("./data/uniswap_v2/burn")]

for i, pool in enumerate(pools):
    if pool in done_pools:
        continue
    response = [0] * 1000

    print(pool, i, len(pools))
    timestamps, block_numbers, sender, to, amount0, amount1 = [], [], [], [], [], []
    from_block, first_timestamp, current_timestamp = 0, 0, 0
    cont = 0

    while len(response) >= 1000 and current_timestamp - first_timestamp < shared.WEEK:
        response = get_response(pool, from_block)
        while response == None or "Max rate limit" in response:
            time.sleep(20)
            response = get_response(pool, from_block)
            if response != None:
                break
        if len(response) == 0:
            continue
        try:
            decoded_logs = clean_response(response)
        except:
            print(pool, i, len(pools))
        # print(decoded_logs[0].keys())
        for k, log in enumerate(decoded_logs):
            # print(log['args'])
            block_numbers.append(log['blockNumber'])
            sender.append(log['args']["sender"])
            to.append(log['args']["to"])
            amount0.append(log['args']["amount0"])
            amount1.append(log['args']["amount1"])
            timestamps.append(int(response[k]["timeStamp"], 16))
        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)

    pd.DataFrame({'timestamps': timestamps, 'block_number': block_numbers, 'sender': sender , 'to': to, 'amount0':amount0, 'amount1':amount1}).to_csv(
        f"./data/uniswap_v2/burn/{pool}.csv", index=False)

0x13bd20ba647E2ed12622851F652a5267869A8c62 0 1459
0x531eB8f8E8e626cCFaB2BAA38d14A45096a50180 1 1459
0x103C0675054D31FD081f53a15FE108c5B2A2f456 2 1459
0x2d8Db3B54bc4c08B295816a30ad74b01109e4e9C 3 1459
0x61637f75DCecEc29e11D73Bfa3ac698Dd5Ef8a0f 4 1459
0xfA69bF4fA125ef9Cc74945EAc914150e5b3cF3E0 5 1459
0xb874c2403391fc1571f778fb6fa9A909eb0E79b7 6 1459
0x4f414E6489fD19bBb50C1Bc6C403E19D0dC71Fd2 7 1459
0x1A3713dE5Fe72EDe1f5aA08d563Da6c5D4Bc2b9e 8 1459
0x53aDd4C98B2787f690042771Ca8E512a5793E9C9 9 1459
0x46c692346CEDCF46C7c9002680E2645c8577EB3a 10 1459
0x99219f505BfffBE1A58fAF113eBc861b175E41f7 11 1459
0x47F4947a1b44F773EDb4520E170621edf40b2f05 12 1459
0xCD3a618948256F77f794FAe7D30Dea18D5ca51A0 13 1459
0xE31461B5adCF2174ae53C64dB57C3F59CfED987b 15 1459
0x36B860b863C23a1564aD19F28FD4260443E007a2 16 1459
0x8D04E60fD7cEd96A24e726968F93397C5a467890 17 1459
0x53f858f28Fd9120bB6Ea1b7F5961333EBE734D22 18 1459
0xDeDef8a9ebd5Fb001A596348EB825f0b9EdaF126 19 1459
0x8B63356fFd04023c8cE3b41C404Fce77DE0f580

In [4]:
import os
import time
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(pool, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={pool}" \
               f"&topic0={sync_hash}" \
               f"&apikey={apikey}"
    try:
        result = json.loads(requests.get(endpoint).text)['result']
    except:
        print(requests.get(endpoint).text)
        result = None
    return result

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Transfer().processReceipt(log_dict)

sync_hash = obtain_hash_event('Transfer(address,address,uint256)')
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
tokens_and_pools = pd.read_csv("../data/uniswap_v2/polygon_pools.csv", index_col='pair')
pools = tokens_and_pools.index.tolist()

contract = web3.eth.contract("0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D", abi=shared.ABI_POOL)
done_pools = [pool.split(".csv")[0] for pool in os.listdir("../data/uniswap_v2/transfer")]

for i, pool in enumerate(pools):
    if pool in done_pools:
        continue
    response = [0] * 1000

    print(pool, i, len(pools))
    timestamps, block_numbers, fromx, to, value = [], [], [], [], []
    from_block, first_timestamp, current_timestamp = 0, 0, 0
    cont = 0

    while len(response) >= 1000 and current_timestamp - first_timestamp < shared.WEEK:
        response = get_response(pool, from_block)
        while response == None or "Max rate limit" in response:
            time.sleep(20)
            response = get_response(pool, from_block)
            if response != None:
                break
        if len(response) == 0:
            continue
        try:
            decoded_logs = clean_response(response)
        except:
            print(pool, i, len(pools))
            # print(response)
        # print(decoded_logs)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            fromx.append(log['args']["from"])
            to.append(log['args']["to"])
            value.append(log['args']["value"])
            timestamps.append(int(response[k]["timeStamp"], 16))
        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)

    pd.DataFrame({'block_number': block_numbers, 'from': fromx , 'to': to, "value":value,  'timestamps': timestamps}).to_csv(
        f"../data/uniswap_v2/transfer/{pool}.csv", index=False)

In [5]:
import os
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(token, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={token}" \
               f"&topic0={transfer_hash}" \
               f"&apikey={apikey}"

    return json.loads(requests.get(endpoint).text)['result']

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Transfer().processReceipt(log_dict)


transfer_hash = obtain_hash_event('Transfer(address,address,uint256)')
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"

huge_tokens = ['0x0000000000000000000000000000000000001010', '0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6',
               '0x692597b009d13C4049a947CAB2239b7d6517875F', '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174',
               '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x3BA4c387f786bFEE076A58914F5Bd38d668B42c3',
               '0xc2132D05D31c914a87C6611C10748AEb04B58e8F', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619']

contract = web3.eth.contract(shared.WMATIC, abi=shared.ABI)
done_transfers = [transfer.split(".csv")[0] for transfer in os.listdir("../data/uniswap_v2/transfer")]
pools = pd.read_csv("../data/uniswap_v2/polygon_pools.csv")
pools = pools.loc[(pools.token0 == shared.WMATIC) | (pools.token1 == shared.WMATIC)]
day = 60 * 60 * 24
tokens = set(pools['token0'].tolist() + pools['token1'].tolist())

for i, token in enumerate(tokens):
    print(token, i, len(tokens))
    if token in done_transfers or token in huge_tokens:
        continue

    first_timestamp = pools.loc[(pools.token0 == token) | (pools.token1 == token)]['timestamp'].iloc[0]
    response = [0] * 1000
    timestamps, block_numbers, from_, to_, value_ = [], [], [], [], []

    from_block, current_timestamp = 0, 0
    cont = 0

    while len(response) >= 1000 and current_timestamp - first_timestamp < day:
        cont += 1
        response = get_response(token, from_block)
        if len(response) == 0:
            break

        decoded_logs = clean_response(response)
        if len(decoded_logs) == 0:
            break

        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            from_.append(log['args']["from"])
            to_.append(log['args']["to"])
            value_.append(log['args']["value"])
            timestamps.append(int(response[k]["timeStamp"], 16))

        if cont == 1:
            first_timestamp = timestamps[0]
        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)

    pd.DataFrame({'block_number': block_numbers, 'from': from_, 'to': to_, 'value': value_, 'timestamps': timestamps}).to_csv(
        f"../data/uniswap_v2/transfer/{token}.csv", index=False)

0x8A311C8CD682e42755f46aB37bF807966D0C3007 0 664
0x7D645CBbCAdE2A130bF1bf0528b8541d32D3f8Cf 1 664
0x42E5E06EF5b90Fe15F853F59299Fc96259209c5C 2 664
0x45a088Dd674bdc4E7454465D47451755B39Fb52c 3 664
0xf29Db13a2EeB469d045aEF04a9C37CA237bbfA72 4 664
0xA3be492848D6cc4992346E786fbB25F4E3c71b4D 5 664
0x2aF1D53393a17B548a07d8ff4CfA873044399fCA 6 664
0xAc988Dd7178218ef4433E76B859D4d990Ed2A309 7 664
0x613a489785C95afEB3b404CC41565cCff107B6E0 8 664
0xaE14b0AFBa354050c58aea291615a6d85bbc5fa3 9 664
0x95f45Ede7b1cF2F944879988f47bC67F51558651 10 664
0x1e7C54F618a6260Fb2eB9Cf4BE995B062A87e2B4 11 664
0xF906695AeAF2aEc43f0343165a909B4A30252F6C 12 664
0xd0170FE488B25d8031cd2e50AE413BB50d329d68 13 664
0x77CBe0689908c544a6856f344352e8B976Cd184a 14 664
0x9c0CBBBc48748b9335f483B085E5092e70E67544 15 664
0x55975321D9d22587d1742DC68B35481C8DBB0Db8 16 664
0xe26cda27c13f4f87cFFc2F437C5900b27eBb5bbB 17 664
0x90B9597bBFDcbc86253d27d6B9a13079c5155f9f 18 664
0xbb71538BB1db7c2C8C5bD78D1b443e440b697d66 19 664
0x4d8e671c